In [43]:
import numpy as np
import pandas as pd
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 300)

In [44]:
target_name = 'is_tansyo'
is_tansyo_predict_result = pd.read_csv("./model/{}_predicted_test.csv".format(target_name), sep=",")

In [45]:
target_name = 'is_hukusyo'
is_hukusyo_predict_result = pd.read_csv("./model/{}_predicted_test.csv".format(target_name), sep=",")

In [46]:
is_tansyo_predict_result.head()

,race_id,frame_number,horse_number,age,burden_weight,horse_weight,date,is_senba,is_mesu,is_osu,distance,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,where_racecourse_中京,where_racecourse_中山,where_racecourse_京都,where_racecourse_函館,where_racecourse_小倉,where_racecourse_新潟,where_racecourse_札幌,where_racecourse_東京,where_racecourse_福島,where_racecourse_阪神,ground_type_ダ,ground_type_芝,is_left_right_straight_右,is_left_right_straight_左,is_left_right_straight_直線,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,where_racecourse_中京_1,where_racecourse_中山_1,where_racecourse_京都_1,where_racecourse_函館_1,where_racecourse_小倉_1,where_racecourse_新潟_1,where_racecourse_札幌_1,where_racecourse_東京_1,where_racecourse_福島_1,where_racecourse_阪神_1,ground_type_ダ_1,ground_type_芝_1,is_left_right_straight_右_1,is_left_right_straight_左_1,is_left_right_straight_直線_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,where_racecourse_中京_2,where_racecourse_中山_2,where_racecourse_京都_2,where_racecourse_函館_2,where_racecourse_小倉_2,where_racecourse_新潟_2,where_racecourse_札幌_2,where_racecourse_東京_2,where_racecourse_福島_2,where_racecourse_阪神_2,ground_type_ダ_2,ground_type_芝_2,is_left_right_straight_右_2,is_left_right_straight_左_2,is_left_right_straight_直線_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,where_racecourse_中京_3,where_racecourse_中山_3,where_racecourse_京都_3,where_racecourse_函館_3,where_racecourse_小倉_3,where_racecourse_新潟_3,where_racecourse_札幌_3,where_racecourse_東京_3,where_racecourse_福島_3,where_racecourse_阪神_3,ground_type_ダ_3,ground_type_芝_3,is_left_right_straight_右_3,is_left_right_straight_左_3,is_left_right_straight_直線_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,rank_4,frame_number_4,horse_number_4,age_4,burden_weight_4,goal_time_4,half_way_rank_4,last_time_4,odds_4,popular_4,horse_weight_4,is_down_4,distance_4,avg_velocity_4,horse_weight_dif_4,burden_weight_rate_4,ground_status_4,total_horse_number_4,is_obstacle_4,weather_rain_4,weather_snow_4,weather_小雨_4,weather_小雪_4,weather_晴_4,weather_曇_4,weather_雨_4,weather_雪_4,where_racecourse_中京_4,where_racecourse_中山_4,where_racecourse_京都_4,where_racecourse_函館_4,where_racecourse_小倉_4,where_racecourse_新潟_4,where_racecourse_札幌_4,where_racecourse_東京_4,where_racecourse_福島_4,where_racecourse_阪神_4,ground_type_ダ_4,ground_type_芝_4,is_left_right_straight_右_4,is_left_right_straight_左_4,is_left_right_straight_直線_4,pre_date_diff_4,is_rider_same_4,is_tamer_same_4,is_owner_same_4,interval_date_4,rank_5,frame_number_5,horse_number_5,age_5,burden_weight_5,goal_time_5,half_way_rank_5,last_time_5,odds_5,popular_5,horse_weight_5,is_down_5,distance_5,avg_velocity_5,horse_weight_dif_5,burden_weight_rate_5,ground_status_5,total_horse_number_5,is_obstacle_5,weath

In [47]:
is_tansyo_predict_result.shape

(97784, 281)

## accuracy の計算
groupbyして、レースごとに確率の最も高い馬を見つける。
正解していたかを判定

In [48]:
# group by したデータフレームに対して行う処理
def check_is_correct_predict_is_tansyo(one_race_data):
    return one_race_data.loc[one_race_data['predict_is_tansyo'].idxmax()]['is_tansyo']
def check_is_correct_predict_is_hukusyo(one_race_data):
    return one_race_data.loc[one_race_data['predict_is_hukusyo'].idxmax()]['is_hukusyo']

In [49]:
is_tansyo_predict_result = is_tansyo_predict_result[["race_id", "predict_is_tansyo", "is_tansyo"]]
is_hukusyo_predict_result = is_hukusyo_predict_result[["race_id","predict_is_hukusyo", "is_hukusyo"]]

is_tansyo = is_tansyo_predict_result.groupby('race_id', as_index=False).apply(check_is_correct_predict_is_tansyo)
is_hukusyo = is_hukusyo_predict_result.groupby('race_id', as_index=False).apply(check_is_correct_predict_is_hukusyo)

In [50]:
print("tansyo accuracy: {}".format(is_tansyo.sum()/is_tansyo.size))
print("hukusyo accuracy: {}".format(is_hukusyo.sum()/is_hukusyo.size))

tansyo accuracy: 0.34874783039920654
tansyo accuracy: 0.7307215472353087


## 回収率の計算
予想値が最大の馬に関して、ある程度のオッズがあれば買う。（オッズが低いと期待値も低いことが多い）

正解していればオッズ分を、不正解なら-100円を出力。あとでそれを回収率のグラフとして見る

In [9]:
predict_result["predict_is_hukusyo"].cumsum()

0         0.000119
1         0.000302
2         0.000400
3         0.000935
4         1.000918
            ...   
195563         NaN
195564         NaN
195565         NaN
195566         NaN
195567         NaN
Name: predict_is_hukusyo, Length: 195568, dtype: float64

In [ ]:
# group by したデータフレームに対して行う処理
def caliculate_money_by_is_tansyo(one_race_data):
    if (one_race_data[one_race_data['predict_is_tansyo'].idxmax()]['rank'] == 1):
        